In [7]:
import pandas as pd
import numpy as np

df=pd.read_csv('radiology_impression_prediction_final.csv')
print ('df size:',df.shape[0])
print (df.columns)

# group by Accession
a_nums=df.Accession.unique()
grouped_dfs=[]
print ('num of unique impressions:',len(a_nums))
for index,num in enumerate(a_nums):
    sub_df=df[df['Accession']==num]
    grouped_dfs.append(sub_df)
    if index!=0 and index%10000==0:
        print('finished set of 10,000 impressions')
print (grouped_dfs[0])

df size: 127683
Index(['Cancer', 'Impression', 'MRN', 'Accession', 'Scan Type', 'Scan Date',
       'Training Set', 'Model1 Prediction', 'Model2 Prediction'],
      dtype='object')
num of unique impressions: 79468
finished set of 10,000 impressions
finished set of 10,000 impressions
finished set of 10,000 impressions
finished set of 10,000 impressions
finished set of 10,000 impressions
finished set of 10,000 impressions
finished set of 10,000 impressions
   Cancer                                         Impression      MRN  \
0       0                               No brain metastases.  2680460   
1       0  Possible post-therapeutic changes to the parot...  2680460   

  Accession        Scan Type  Scan Date  Training Set  Model1 Prediction  \
0   5399656  CT INFUSED HEAD  9/30/2003          True           0.041939   
1   5399656  CT INFUSED HEAD  9/30/2003          True           0.000488   

   Model2 Prediction  
0                NaN  
1                NaN  


In [10]:
### re-create impressions using grouped df
# info to include: MRN, Accession, Scan Date, Scan Type, Impression (re-built), Cancer
data=[]
for index,sub_df in enumerate(grouped_dfs):
    MRN=sub_df.iloc[0]['MRN']
    Accession=sub_df.iloc[0]['Accession']
    date=sub_df.iloc[0]['Scan Date']
    scan_type=sub_df.iloc[0]['Scan Type']
    impression=''
    cancer_status=0
    ### only have cancer if Cancer=1 for some line of that impression
    if 1 in sub_df.Cancer.unique():
        cancer_status=1
    for index,row in sub_df.iterrows():
        impression=impression+row['Impression']+' '
    datum=[MRN,Accession,date,scan_type,impression,cancer_status]
    data.append(datum)
df_to_export=pd.DataFrame(data=data,columns=['mrn','accession','enc_date','scan_type','impression','cancer_status'])
df_to_export.to_csv('examine.csv',index=False)

In [13]:
### convert enc_date to DateTime
df=pd.read_csv('examine.csv')
df['enc_date']=pd.to_datetime(df['enc_date'])
df=df.sort_values(by='enc_date')

### group by mrn
mrns=df.mrn.unique()
ptx_info=[]
for mrn in mrns:
    sub_df=df[df['mrn']==mrn]
    ptx_info.append(sub_df)
df_test=ptx_info[0]

See body of report                                                    Attendng: Peter Jokich, MD                                                                  Fellow:   Arthur Stillman, MD                                                               Resident:   Technicn:   Transcbr:  0
No intracranial hemorrhage. No brain masses. No masses or lymphadenopathy within the neck. Post-therapy changes in the right neck. Chronic maxillary sinusitis. Chronic mastoiditis on the left.    0
No intracranial hemorrhage. No brain masses. No masses or lymphadenopathy within the neck. Post-therapy changes in the right neck. Chronic maxillary sinusitis. Chronic mastoiditis on the left.    0
 No evidence of intrathoracic or upper abdominal primary or metastatic neoplasm.  Clear lungs, except for minimal and localized left apical post-infectious or post-radiation scarring, and no evidence of active lung disease.  Normal heart size and pulmonary perfusion.  A small right renal cyst is insignificant.  

In [19]:
total_df=pd.DataFrame()
for i,ptx in enumerate(ptx_info):
    is_RM=[]
    has_cancer=False
    for index,row in ptx.iterrows():
        if row['cancer_status']==1:
            is_RM.append(1)
            has_cancer=True
        elif row['cancer_status']==0 and has_cancer:
            is_RM.append(1)
        elif row['cancer_status']==0 and not has_cancer:
            is_RM.append(0)
    ptx_info[i]['is_RM']=is_RM
    total_df=total_df.append(ptx_info[i])
#print (total_df)
df_to_export=total_df[['enc_date','mrn','is_RM']]
df_to_export.columns=['date_of_service_dttm','mrn','is_RM']
df_to_export.to_csv('RM_MCs_radiology.csv',index=False)

C:\Users\12488\python\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
